In [1]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [4]:
def tutorial3():
    col = IntegerAttr.from_int_and_width(1, i32)
    row = IntegerAttr.from_int_and_width(4, i32)
    tile14 = aie.TileOp(col, row)

    col = IntegerAttr.from_int_and_width(2, i32)
    row = IntegerAttr.from_int_and_width(4, i32)
    tile24 = aie.TileOp(col, row)

    shape = ArrayAttr[i32]([IntegerAttr.from_int_and_width(256, i32)])
    buf_sym_name = StringAttr("a24")
    buf = aie.BufferOp(tile24, i32, shape, buf_sym_name)

    lock_id1 = IntegerAttr.from_int_and_width(1, i32)
    lock_id2 = IntegerAttr.from_int_and_width(2, i32)
    lock_init = IntegerAttr.from_int_and_width(0, i32)

    lock24_1 = aie.LockOp(lock_id1, lock_init, tile24, StringAttr("lock_a24_1"))
    lock24_2 = aie.LockOp(lock_id2, lock_init, tile24, StringAttr("lock_a24_2"))

    value_0 = IntegerAttr.from_int_and_width(0, i32)
    value_1 = IntegerAttr.from_int_and_width(1, i32)
    blocking = IntegerAttr.from_int_and_width(1, i32)
    acquire = IntegerAttr.from_int_and_width(1, i32)
    release = IntegerAttr.from_int_and_width(0, i32)

    @Builder.region
    def region0(builder : Builder):
        uselock24_1_acquire = aie.UseLockOp(value_0, acquire, blocking, lock24_1)
        uselock24_1_release = aie.UseLockOp(value_1, release, blocking, lock24_1)
        val = arith.Constant.from_int_and_width(14, i32)
        idx = arith.Constant.from_int_and_width(3, IndexType())
        builder.insert(uselock24_1_acquire)
        memref_store = memref.Store.get(val, buf, idx)
        builder.insert(val)
        builder.insert(idx)
        builder.insert(memref_store)
        builder.insert(uselock24_1_release)
        builder.insert(aie.EndOp())

    stackSize = IntegerAttr.from_int_and_width(1, i32)
    core14 = aie.CoreOp(stackSize, tile14, region0)

    @Builder.region
    def region1(builder : Builder):
        uselock24_2_acquire = aie.UseLockOp(value_0, acquire, blocking, lock24_2)
        uselock24_1_acquire = aie.UseLockOp(value_1, acquire, blocking, lock24_1)
        
        idx1 = arith.Constant.from_int_and_width(3, IndexType())
        d1 = memref.Load.get(buf, idx1)
        c1 = arith.Constant.from_int_and_width(100, i32)
        d2 = arith.Addi(d1, c1)
        idx2 = arith.Constant.from_int_and_width(5, IndexType())
        memref_store = memref.Store.get(d2, buf, idx2)

        uselock24_2_release = aie.UseLockOp(value_1, release, blocking, lock24_2)

        builder.insert(uselock24_2_acquire)
        builder.insert(uselock24_1_acquire)
        builder.insert(idx1)
        builder.insert(d1)
        builder.insert(c1)
        builder.insert(d2)
        builder.insert(idx2)
        builder.insert(memref_store)
        builder.insert(uselock24_2_release)
        builder.insert(aie.EndOp())

    stackSize = IntegerAttr.from_int_and_width(1, i32)
    core24 = aie.CoreOp(stackSize, tile24, region1)

    module = builtin.ModuleOp([tile14, tile24, buf, lock24_1, lock24_2, core14, core24])

    print(module)

In [5]:
tutorial3()

builtin.module {
  %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 4 : i32} : () -> index
  %1 = "AIE.tile"() {"col" = 2 : i32, "row" = 4 : i32} : () -> index
  %2 = "AIE.buffer"(%1) {"sym_name" = "a24"} : (index) -> memref<256xi32>
  %3 = "AIE.lock"(%1) {"lockID" = 1 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_1"} : (index) -> index
  %4 = "AIE.lock"(%1) {"lockID" = 2 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_2"} : (index) -> index
  %5 = AIE.core (%0) {
    AIE.useLock(%3, "Acquire", 0)
    %6 = arith.constant 14 : i32
    %7 = arith.constant 3 : index
    memref.store %6, %2[%7] : memref<256xi32>
    AIE.useLock(%3, "Release", 1)
    "AIE.end"() : () -> ()
  }
  %8 = AIE.core (%1) {
    AIE.useLock(%4, "Acquire", 0)
    AIE.useLock(%3, "Acquire", 1)
    %9 = arith.constant 3 : index
    %10 = memref.load %2[%9] : memref<256xi32>
    %11 = arith.constant 100 : i32
    %12 = arith.addi %10, %11 : i32
    %13 = arith.constant 5 : index
    memref.store %12, %2[%13] : memref<256xi